# A minimal standard for interoperable Schemas

The **minimal standard** is defined as a Protocol, with some Abstract Base Classes. The protocol is minimal in that it provides for the most common [Use Cases](https://github.com/mcgfeller/py-schemas/blob/master/UseCases.md):
* Serialization to an external representation
* Deserialization from an external representation
* Validation
* Attach a Schema to an object class
* Obtaining Schema elements
* Static type checking
  * Get the Python type of a schema element 
* Minimal Schema transformation:
  * Get [dataclasses.Field](https://docs.python.org/3/library/dataclasses.html#dataclasses.Field) of a schema element
  * Construct a a Schema element from dataclasses.Field
  * Construct a Schema from another Schema (from another Schema solution) by going through dataclasses.Field for each element. 
* Associate data with Schema

*The protocol doesn't provide a standard representation for Schemas or Schema Elements; it only provides standard access and use.* It does provide minimal conversion of arbitrary Schema features between schema libraries, as it provides conversion to Python static types and [dataclasses.Field](https://docs.python.org/3/library/dataclasses.html#dataclasses.Field). See [Alternatives considered](alternatives.md).

Using the Protocol for a single schema library, such as **[Marshmallow](https://marshmallow.readthedocs.io/en/latest/)**, does not provide facilities superior to native usage. However, if the protocol is implemented by several libraries, integration of libraries using different schema facilities becomes much easier. The [Marshmallow ecosystem](https://github.com/marshmallow-code/marshmallow/wiki/Ecosystem) lists a few integrations to other libraries using Schemas (JSON, dataclasses, SQL Alchemy), each being a once-off ad-hoc solution.


In [1]:
import abc
import typing
import collections.abc
import enum
import dataclasses

First, an entirely optional, a class providing a protocol to associate a Schema with a class, without constraining too much how things are composed. We only mandate that if a Schema is assigned to a subclass of `SchemedObject`, the Schema can be retrieved by `.__get_schema__()`. 

In [2]:
class SchemedObject(metaclass=abc.ABCMeta):
    """ An object with a Schema, supporting the __get_schema__ method.
    """

    @classmethod
    @abc.abstractmethod    
    def __get_schema__(cls) -> 'AbstractSchema':
        pass

To standardize on representations into which we serialize, we enumerate a few, so we can use them as enum. 

In [3]:
class WellknownRepresentation(enum.Enum):

    python  = '__python__' # internal python structures
    pickle  = 'application/python-pickle'
    json    = 'application/json'
    xml     = 'application/xml'
    sql     = 'application/sql'
    html    = 'text/html'

The minimal protocol for a Schema defines it as an Iterable, yielding Schema Elements, having a set of representations it supports, and methods to convert to and from external representations. 

We want the Schema to be useful for type declarations, so `.get_annotations()` and `.as_field_annotations()` return dictionaries usable as `__annotations__` in a class. Unfortunately, the Python `typing` module assumes `__annotations__` to be a dictionary, instead of allowing a callable returning annotations, so I don't think this can be done more elegantly. 

We also have a `.get_metadata()` method defined both on the Schema and the Schema Element. Metadata is not used at all by the Schema, and is provided as a third-party extension mechanism. Multiple third-parties can each have their own key, to use as a namespace in the metadata. This is similar to and taken from [dataclasses.Field](https://docs.python.org/3/library/dataclasses.html#dataclasses.field).

The `writer_callback` argument in `.to_external()` and the `external` in `.from_external()` are inspired by the consumer API of [PEP-574](https://www.python.org/dev/peps/pep-0574/#consumer-api). The support arbitrary callables to receive or source the external representation, respectively; if not supplied, the external representation is returned or input as a string argument, respectively. 

The optional methods `.from_schema()` and `.add_element()` support the creation of a Schema and adding elements from another Schema (from another solution). They must not assume any representation of the source schema, apart from the one exposed in this protocol. 

In [4]:
class AbstractSchema(collections.abc.Iterable, metaclass=abc.ABCMeta):
    """ The AbstractSchema does not prescribe how the Schema is organized, and
        only prescribes that the AbstractSchemaElement may be obtained by iterating
        over the Schema.
    """

    SupportedRepresentations: typing.ClassVar[typing.Set["WellknownRepresentation"]] = {
        WellknownRepresentation.python
    }

    @abc.abstractmethod
    def to_external(
        self,
        obj: SchemedObject,
        destination: WellknownRepresentation,
        writer_callback: typing.Optional[typing.Callable] = None,
        **params,
    ) -> typing.Optional[typing.Any]:
        """
            If *writer_callback* is None (the default), the external representation
            is returned as result.

            If *writer_callback* is not None, then it can be called any number
            of times with some arguments. No result is returned.

            (inspired by PEP-574 https://www.python.org/dev/peps/pep-0574/#producer-api)
        """
        pass

    @abc.abstractmethod
    def from_external(
        self,
        external: typing.Union[typing.Any, typing.Callable],
        source: WellknownRepresentation,
        **params,
    ) -> SchemedObject:

        """
            If *external* is bytes, they are consumed as source representation.

            If *external* is a Callable, then it can be called any number
            of times with some arguments to obtain parts of the source representation.

        """
        pass

    @abc.abstractmethod
    def validate_internal(self, obj: SchemedObject, **params) -> SchemedObject:
        pass

    @abc.abstractmethod
    def __iter__(self) -> typing.Iterator["AbstractSchemaElement"]:
        """ iterator through SchemaElements in this Schema """
        pass

    def get_annotations(self) -> typing.Dict[str, typing.Type]:
        """ return Schema Elements in annotation format.
            Use as class.__annotations__ = schema.get_annotations()
            I would wish that __annotations__ is a protocol that can be provided,
            instead of simply assuming it is a mapping. 
        """
        return {se.get_name(): se.get_python_type() for se in self}

    def as_field_annotations(self) -> typing.Dict[str, dataclasses.Field]:
        """ return Schema Elements in DataClass field annotation format.
            Use as class.__annotations__ = schema.as_field_annotations().
        """
        return {se.get_name(): se.get_python_field() for se in self}

    def get_metadata(self) -> typing.Mapping[str, typing.Any]:
        """ return metadata (aka payload data) for this Schema.

            Meta data is not used at all by the Schema, and is provided as a third-party 
            extension mechanism. Multiple third-parties can each have their own key, 
            to use as a namespace in the metadata.
            (similar to and taken from dataclasses.Field)

            Can be refined; by default an empty dict is returned.

            There is a similar method defined on the AbstractSchemaElement for
            smetadata attached to a schema element. 
        """
        return {}

    @classmethod
    @abc.abstractmethod
    def from_schema(cls, schema: "AbstractSchema") -> "AbstractSchema":
        """ Optional API: create a new Schema (in the Schema dialect of the cls) from
            a schema in any Schema Dialect.
        """
        pass

    @abc.abstractmethod
    def add_element(self, element: "AbstractSchemaElement"):
        """ Optional API: Add a Schema element (in any Schema Dialect) to this Schema. 
        """
        pass


We finish the minimal protocol by defining a Schema Element. It supports a back reference to its Schema, and two methods to get the Schema Element's name and its Python type (for interoperability with Python static typing).

The optional method `.from_schema_element()` supports the creation of a SchemaElement and adding elements from another SchemaElement (from another solution). They must not assume any representation of the source SchemaElement, apart from the one exposed in this protocol. 

*Implementation hints:* 
 - Use `.get_python_field()` to retrieve the data from a SchemaElement, and construct a SchemaElement from the Field.
 - If argument `schema_element` is from your Schema solution, you may return it unchanged. 


In [5]:
class AbstractSchemaElement(metaclass=abc.ABCMeta):
    @abc.abstractmethod
    def get_schema(self) -> typing.Optional[AbstractSchema]:
        """ get associated schema or None """
        pass

    @abc.abstractmethod
    def get_name(self) -> str:
        """ get name useable as variable name """
        pass

    @abc.abstractmethod
    def get_python_type(self) -> type:
        """ get Python type of this AbstractSchemaElement """
        pass

    @abc.abstractmethod
    def get_python_field(self) -> dataclasses.Field:
        """ get Python dataclasses.Field corresponding to AbstractSchemaElement.
            Unless refined, just packs type into a Field and attaches metadata. 
        """
        dcfield = dataclasses.field(metadata=self.get_metadata())
        dcfield.type = self.get_python_type()
        return dcfield

    def get_metadata(self) -> typing.Mapping[str, typing.Any]:
        """ return metadata (aka payload data) for this SchemaElement.

            Meta data is not used at all by the Schema, and is provided as a third-party 
            extension mechanism. Multiple third-parties can each have their own key, 
            to use as a namespace in the metadata.
            (similar to and taken from dataclasses.Field)

            Can be refined; by default an empty dict is returned.
        """
        return {}

    @classmethod
    @abc.abstractmethod
    def from_schema_element(
        cls, schema_element: "AbstractSchemaElement"
    ) -> "AbstractSchemaElement":
        """ Optional API: create a new AbstractSchemaElement (in the Schema dialect of the cls) from
            a AbstractSchemaElement in any Schema Dialect.
        """
        pass




We now continue by adapting a well-know Schema library. 



## Marshmallow 

I use [Marshmallow](https://marshmallow.readthedocs.io/en/latest/) as a Schema library. Marshmallow is a full-featured, well-engineered, stand-alone library supporting serialization to Python natives types and JSON. 

### Adapting the Schema for Marshmallow

I assume the above code is available as module `abc_schema`. I don't change the source of Marshmallow, I use subclassing for the Schema and monkey-patching for the fields.

The adaption to _Nested_ fields would be trickier, but would follow the same principles (personally, I prefer using an field _Object_, which refers to another schemed class, and works with `post_load()` to re-create the object).  

In [6]:
""" Marshmallow based conformant Schema.
    Marshmallow fields are monkey-patched.
    Marshmallow schema is subclassed. 
"""

import typing
import marshmallow as mm # type: ignore
import decimal
import datetime
import abc_schema
import dataclasses

Now adapt Marshmallow fields by monkey patching:

In [7]:
""" Methods for Marshmallow fields (will be monkey-patched) """


def get_schema(self) -> typing.Optional["MMSchema"]:
    """ return the Schema or None """
    return self.root


def get_name(self) -> str:
    return self.name


def get_python_type(self) -> typing.Type:
    """ get native type of field. 

    """
    return FieldType_to_PythonType.get(self.__class__, typing.Type[typing.Any])


def get_python_field(self) -> dataclasses.Field:
    """ get Python dataclasses.Field corresponding to SchemaElement.
        Fills default if provided in field. If field is not required and there is no default, make type Optional.
    """
    pytype = self.get_python_type()
    # mm.missing -> dataclasses.MISSING>
    default = dataclasses.MISSING if self.missing is mm.missing else self.missing
    
    if (not self.required) and default is dataclasses.MISSING:
        pytype = typing.Optional[pytype]  # type: ignore # I don't understand mypy's problem here!
    dcfield = dataclasses.field(default=default, metadata=self.get_metadata())
    dcfield.type = pytype
    return dcfield


def get_metadata(self) -> typing.Mapping[str, typing.Any]:
    """ return metadata (aka payload data) for this SchemaElement.
    """
    return self.metadata


def from_schema_element(cls, schema_element: abc_schema.AbstractSchemaElement) -> mm.fields.Field:
    """ Classmethod: Create a new Marshmallow Field from
        a AbstractSchemaElement in any Schema Dialect.

        In a real implementation, we could return schema_element unchanged
        if isinstance(schema_element,mm.fields.Field). However, we only
        rely on the protocol API here. 

    """
    pf = schema_element.get_python_field()
    pt = pf.type
    required = True

    # Determine whether pt is an Optional type, which is a Union[pt,None] 
    # typing has very limited inspection features: 
    if getattr(pt, "__origin__", None) is typing.Union:  # is this typing.Union?
        if len(pt.__args__) == 2 and pt.__args__[1] in (
            None,
            None.__class__,
        ):  # Optional type
            pt = pt.__args__[0]  # actual type
            required = False # optional means it's not required

    mmf = from_python_type(pt, required, pf.default, pf.metadata)
    if mmf:
        return mmf
    else:
        raise ValueError(
            f"Cannot determine Marshmallow field for dataclassed.Field {pf} with type {pt} in element {schema_element}"
        )


def from_python_type(
    pt: type, required: bool = True, default: typing.Any = dataclasses.MISSING, 
    metadata: typing.Mapping[str, typing.Any] = None) -> typing.Optional[mm.fields.Field]:
    """ Create a new Marshmallow Field from a python type, either type, class, or typing.Type.
        We first check the special _name convention for typing.Type, 
        then check whether the FieldType has a _type_factory or is constructed by its class.
    """
    pt_name = getattr(pt, "_name", None)
    if pt_name:
        field_class = TypingName_to_FieldType.get(pt_name)
    else:
        field_class = None
    if not field_class:
        field_class = PythonType_to_FieldType.get(pt)
        if not field_class:
            return None

    if default is dataclasses.MISSING:  # dataclasses.MISSING ->  mm.missing
        default = mm.missing

    type_factory = getattr(field_class, "_type_factory", None)
    if type_factory:
        mmf = type_factory(pt, required=required, default=default, metadata=metadata)
    else:
        mmf = field_class(
            required=required, missing=default, default=default, metadata=metadata
        )
    return mmf


# monkey-patch all Fields:
mm.fields.Field.get_schema = get_schema
mm.fields.Field.get_name = get_name
mm.fields.Field.get_python_type = get_python_type
mm.fields.Field.get_python_field = get_python_field
mm.fields.Field.get_metadata = get_metadata
mm.fields.Field.from_schema_element = classmethod(from_schema_element)

FieldType_to_PythonType: typing.Dict[mm.fields.FieldABC, typing.Type] = {
    mm.fields.Integer:          int,
    mm.fields.Float:            float,
    mm.fields.Decimal:          decimal.Decimal,
    mm.fields.Boolean:          bool,
    mm.fields.Email:            str,    
    mm.fields.FormattedString:  str,
    mm.fields.Str:              str, # least specific last
    mm.fields.DateTime:         datetime.datetime,
    mm.fields.Time:             datetime.time,
    mm.fields.Date:             datetime.date,
    mm.fields.TimeDelta:        datetime.timedelta,
    mm.fields.Dict:             typing.Dict,
}

# reverse list, least specific overwrites most specific:
PythonType_to_FieldType = {pt: ft for ft, pt in FieldType_to_PythonType.items()}



Dict as a container field with fields for keys and values is a bit more involved, so we hook methods to get the Python type based on the container types, and a factory to create a Marshmallow dict from the composed Python type.

The introspection facilities of the Python `typing` module seem limited, so I have to use some internals. See [Typing Inspect](https://github.com/ilevkivskyi/typing_inspect) for an alternative. 

In [8]:
# Types from typing have a _name field - I found no other way to determine what typing.Dict actually is:
TypingName_to_FieldType: typing.Dict[str, mm.fields.FieldABC] = {
    'Dict':                     mm.fields.Dict,
}


def _dict_get_python_type(self) -> type:
    """ get native classes of containers and build Dict type
        Simplified - either container is a Field, or we use Any.
    """
    kt = (
        self.key_container.get_python_type()
        if isinstance(self.key_container, mm.fields.FieldABC)
        else typing.Type[typing.Any]
    )
    vt = (
        self.value_container.get_python_type()
        if isinstance(self.value_container, mm.fields.FieldABC)
        else typing.Type[typing.Any]
    )
    return typing.Dict[kt, vt]  # type: ignore # mypy cannot handle this dynamic typing without a plugin!


def _dict_type_factory(
    cls, pt: typing.Type, required: bool, default: typing.Any, metadata: dict
) -> mm.fields.Field:
    """ get MM fields.Dict from Python type. 
        get key class and value class (both can be None), then construct Dict.
    """
    kc = from_python_type(pt.__args__[0])
    vc = from_python_type(pt.__args__[1])
    return cls(
         keys=kc, values=vc, required=required, missing=default, default=default, metadata=metadata
    )


mm.fields.Dict.get_python_type = _dict_get_python_type
mm.fields.Dict._type_factory = classmethod(_dict_type_factory)



The SchemedObject can be used a superclass for Python objects with an associated Schema. By convention of this implementation, the Schema is obtained as an inner class named Schema.

The Schema is instantiated and cached as `.__schema`. It also sets `__objclass__` in the schema of a class to that class, so `MMSchema.object_factory()` can recreate the actual class instance. I prefer this over a Marshmallow `post_load()` decorator, because that needs to be defined on the Schema (however, this is not part of the protocol, but of my adaption of Marshmallow). 

In [9]:
class SchemedObject:
    """ SchemedObject is the - entirely optional - superclass that can be used for classes that have an associated
        Schema. It defines one class method .__get_schema__, to return that Schema.

        By convention of this implementation, the Schema is obtained as an inner class named Schema.
        The Schema is instantiated and cached as .__schema. __objclass__ is set in the Schema so
        that .object_factory() can create an instance. 
    """

    @classmethod
    def __get_schema__(cls):
        """ get schema attached to class, and cached in cls.__schema. If not cached, instantiate .Schema """
        s = getattr(cls, "__schema", None)
        if s is None:
            sclass = getattr(cls, "Schema", None)
            if sclass is None:
                raise ValueError("Class must have Schema inner class")
            else:
                s = cls.__schema = sclass()  # instantiate
                s.__objclass__ = cls  # assign this class to schema.__objclass__
        return s


abc_schema.SchemedObject.register(SchemedObject)

__main__.SchemedObject

Now for the Schema itself, and its transformation methods:

In [10]:
class MMSchema(mm.Schema):

    SupportedRepresentations = {
        abc_schema.WellknownRepresentation.python,
        abc_schema.WellknownRepresentation.json,
    }

    def to_external(
        self,
        obj: SchemedObject,
        destination: abc_schema.WellknownRepresentation,
        writer_callback: typing.Optional[typing.Callable] = None,
        **params,
    ) -> typing.Optional[typing.Any]:
        """
            If *writer_callback* is None (the default), the external representation
            is returned as result.

            If *writer_callback* is not None, then it can be called any number
            of times with some arguments. No result is returned.

            (inspired by PEP-574 https://www.python.org/dev/peps/pep-0574/#producer-api)
        """
        supported = {
            abc_schema.WellknownRepresentation.json: self.dumps,
            abc_schema.WellknownRepresentation.python: self.dump,
        }
        method = supported.get(destination)
        if not method:
            raise ValueError(f"destination {destination} not supported.")
        e = method(obj, **params)
        if writer_callback:
            return writer_callback(e)
        else:
            return e

    def from_external(
        self,
        external: typing.Union[typing.Any, typing.Callable],
        source: abc_schema.WellknownRepresentation,
        **params,
    ) -> typing.Union[SchemedObject, typing.Dict[typing.Any, typing.Any]]:

        """
            If *external* is bytes, they are consumed as source representation.

            If *external* is a Callable, then it can be called any number
            of times with some arguments to obtain parts of the source representation.

        """
        supported = {
            abc_schema.WellknownRepresentation.json: self.loads,
            abc_schema.WellknownRepresentation.python: self.load,
        }
        method = supported.get(source)
        if not method:
            raise ValueError(f"source {source} not supported.")
        if callable(external):
            external = external(None)
        d = method(external, **params)
        o = self.object_factory(d)

        return o

    def validate_internal(self, obj: SchemedObject, **params) -> SchemedObject:
        """ Marshmallow doesn't provide validation on the object - we need to dump it.
            As Schema.validate returns a dict, but we want an error raised, we call .load() instead.
            However, if the validation doesn't raise an error, we return the argument obj unchanged. 
        """
        dummy = self.load(self.dump(obj))  # may raise an error
        return obj

    def __iter__(self):
        """ iterator through SchemaElements in this Schema, sett """
        for name, field in self._declared_fields.items():
            field.name = name
            yield field

    def object_factory(self, d: dict) -> typing.Union[SchemedObject, dict]:
        """ return an object from dict, according to the Schema's __objclass__ """
        objclass = getattr(self, "__objclass__", None)
        if objclass:
            o = objclass(**d)  # factory!
        else:
            o = d
        return o

    def get_metadata(self) -> typing.MutableMapping[str, typing.Any]:
        """ return metadata (aka payload data) for this Schema.
            Meta data is not used at all by the Schema, and is provided as a third-party 
            extension mechanism. Multiple third-parties can each have their own key, 
            to use as a namespace in the metadata (similar to and taken from dataclasses.Field)
        """
        return self.context

    def get_annotations(self) -> typing.Dict[str, typing.Type]:
        """ return Schema Elements in annotation format.
            same as in abc_schema, but cannot inherit (cannot subclass due to metaclass conflict).
        """
        return {se.get_name(): se.get_python_type() for se in self}

    def as_field_annotations(self) -> typing.Dict[str, dataclasses.Field]:
        """ return Schema Elements in DataClass field annotation format. 
            same as in abc_schema, but cannot inherit (cannot subclass due to metaclass conflict).
        """
        return {se.get_name(): se.get_python_field() for se in self}

    @classmethod
    def from_schema(cls, schema: abc_schema.AbstractSchema) -> "MMSchema":
        """ Create a new Marshmallow Schema from a schema in any Schema Dialect.
            Unfortunately, Marshmallow has no API to add fields, so we use internal APIs. 
            See https://github.com/marshmallow-code/marshmallow/issues/1201.
        """
        s = MMSchema(context=schema.get_metadata())  # base Schema
        # add fields
        s.declared_fields = {
            element.get_name(): mm.fields.Field.from_schema_element(element)
            for element in schema
        }
        s.fields = s._init_fields()  # invoke internal API to bind fields 
        return s

    def add_element(self, element: abc_schema.AbstractSchemaElement):
        """ Add a Schema element to this Schema.
            We're afraid to use internal API to add additional fields.
            See https://github.com/marshmallow-code/marshmallow/issues/1201.
            This API is optional, after all.
        """
        raise NotImplementedError("Marshmallow API doesn't support adding fields")


abc_schema.AbstractSchema.register(MMSchema)


__main__.MMSchema

### Examples

Using the above code, we declare a Person class with a Marshmallow Schema:

In [11]:
from marshmallow_schema import SchemedObject, MMSchema
import abc_schema
import marshmallow as mm  # type: ignore
import dataclasses
import datetime
import typing

In [12]:
class Person(SchemedObject):
    class Schema(MMSchema):
        name = mm.fields.Str(required=True)
        email = mm.fields.Email(missing=None)
        dob = mm.fields.Date(required=False,missing=None)
        sex = mm.fields.Str(
            validate=mm.fields.validate.OneOf(("m", "f", "o", "?")), missing="?"
        )
        education = mm.fields.Dict(
            keys=mm.fields.Str(), values=mm.fields.Date(), payload="field metadata"
        )

    __annotations__ = Schema().get_annotations()


p = Person()

In [13]:
{se.get_name(): se.get_python_type() for se in p.__get_schema__()}

{'name': str,
 'email': str,
 'dob': datetime.date,
 'sex': str,
 'education': typing.Dict[str, datetime.date]}

In [14]:
s = p.__get_schema__()
assert s.fields["name"].get_schema() is s
assert s.fields["education"].get_metadata() == {"payload": "field metadata"}

`education` is a Marshmallow field:

In [15]:
field = s.fields["education"]
field

<fields.Dict(default=<marshmallow.missing>, attribute=None, validate=None, required=False, load_only=False, dump_only=False, missing=<marshmallow.missing>, allow_none=False, error_messages={'required': 'Missing data for required field.', 'null': 'Field may not be null.', 'validator_failed': 'Invalid value.', 'invalid': 'Not a valid mapping type.'})>

Use it to create another Marshmallow field without using field's Marshmallow internals:

In [16]:
mm.fields.Field.from_schema_element(field)

<fields.Dict(default=<marshmallow.missing>, attribute=None, validate=None, required=False, load_only=False, dump_only=False, missing=<marshmallow.missing>, allow_none=False, error_messages={'required': 'Missing data for required field.', 'null': 'Field may not be null.', 'validator_failed': 'Invalid value.', 'invalid': 'Not a valid mapping type.'})>

Create another Marshmallow schema from s, only using the protocol API to access s (round trip): 

In [17]:
scopy = MMSchema.from_schema(s)

We can also define a data class:

In [18]:
@dataclasses.dataclass
class DCPerson(Person):
     __annotations__ = Person.Schema().as_field_annotations()

Note we need to set dob=None, as dataclasses cannot handle optional fields, but since the type is optional, it's still a valid value:

In [19]:
dcp = DCPerson(name='Martin',email='mgf@acm.org',sex='m',dob=None,education={"Gymnasium Raemibuehl": datetime.date(1981, 9, 1)})

In [20]:
dcp

DCPerson(name='Martin', email='mgf@acm.org', dob=None, sex='m', education={'Gymnasium Raemibuehl': datetime.date(1981, 9, 1)})

In [21]:
dcp_s = dcp.__get_schema__()

In [22]:
j = dcp_s.to_external(dcp,abc_schema.WellknownRepresentation.json)

In [23]:
j

'{"sex": "m", "dob": null, "email": "mgf@acm.org", "education": {"Gymnasium Raemibuehl": "1981-09-01"}, "name": "Martin"}'

In [24]:
o = dcp_s.from_external(j,abc_schema.WellknownRepresentation.json)

In [25]:
dcp_s.validate_internal(dcp)

DCPerson(name='Martin', email='mgf@acm.org', dob=None, sex='m', education={'Gymnasium Raemibuehl': datetime.date(1981, 9, 1)})

Validate with the "copy" of the Schema created above using only the protocol API:

In [26]:
scopy.validate_internal(dcp)

DCPerson(name='Martin', email='mgf@acm.org', dob=None, sex='m', education={'Gymnasium Raemibuehl': datetime.date(1981, 9, 1)})

### MyPy static type checking

Note that MyPy currently cannot check type definitions returned by functions and methods (definitions, not the return type itself). However, MyPy has a [plugin mechanism](http://mypy-lang.blogspot.com/2019/03/extending-mypy-with-plugins.html) that should support such calculated types. 

In [27]:
!mypy marshmallow_example.py

### Conclusion

Using Marshmallow through the protocol is not easier than using the standard Marshmallow API. However, if in a Marshmallow 
Schema one needs to refer to some Schemas in Django or SQLAlchemy, interoperability becomes key.